In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## Scrapping Data

In [2]:
product_links = []
for i in range (1,23):
    if(i==1):
        url = "https://www.bonkerscorner.com/product-category/men/"
        r = requests.get(url)
        htmlContent = r.content
        soup = BeautifulSoup(htmlContent,'html.parser')
        products = soup.find_all('div',class_ = "rey-productInner")
        for product in products:
            title_div = product.find("div", class_="rey-productThumbnail")

            product_links.append(title_div.find('a').get('href'))
    else:
        url = f"https://www.bonkerscorner.com/product-category/men/page/{i}/"
        r = requests.get(url)
        htmlContent = r.content
        soup = BeautifulSoup(htmlContent,'html.parser')
        products = soup.find_all('div',class_ = "rey-productInner")
        for product in products:
            title_div = product.find("div", class_="rey-productThumbnail")

            product_links.append(title_div.find('a').get('href'))
        

In [3]:
len(product_links)

343

In [4]:
keys = ['Composition', 'GSM', 'Colour']
patterns = [r'Composition:\s*(.*)', r'GSM:\s*(.*)', r'Colour:\s*(.*)|Color:\s*(.*)']

In [9]:
d = {'link':[], 'title':[], 'OG price':[], 'DC price':[], 'Composition':[], 'GSM':[], 'Colour':[]}

In [10]:
for link in product_links:
    url = link
    r = requests.get(url)
    htmlContent = r.content
    soup = BeautifulSoup(htmlContent,'html.parser')
    title = soup.find('div',class_="rey-innerSummary")
    desc = soup.find('div',class_="rey-wcPanel-inner")
    
    d['link'].append(link)
    d['title'].append(title.find('h1').get_text())
    d['OG price'].append(title.find('del').get_text())
    d['DC price'].append(title.find('ins').get_text())
    
    res = desc.get_text()
    
    for key, pattern in zip(keys, patterns):
        match = re.search(pattern, res)
        d[key].append(match.group(1) if match else '')

In [11]:
bonkers_df = pd.DataFrame.from_dict(d)

In [12]:
bonkers_df.to_csv('bonkers.csv', index=False)